DL_Assignment_Part_B

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from torchvision import models

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define function for pretrain model
def pretrain_model(model_name, freeze_percent,freez_all_except_last_layer):
    
    
    if model_name == 'resnet':
        pretrained_model = models.resnet50(pretrained=True)
    elif model_name == 'inception':
        pretrained_model = models.inception_v3(pretrained=True)
   
    nodes_fc = pretrained_model.fc.in_features
    pretrained_model.fc = nn.Linear(nodes_fc, 10)  # Change 10 to the number of classes

    total_layers = sum(1 for _ in pretrained_model.children())
    num_freeze = int(total_layers * freeze_percent)

    if freez_all_except_last_layer == 'Yes':
        for name, parameter in pretrained_model.named_parameters():
            if not name.startswith('fc'):  # Exclude the last layer named 'fc'
                parameter.requires_grad = False

    if freez_all_except_last_layer == 'No':
        count = 0
        for name, name_chld in pretrained_model.named_children():
            if count < num_freeze:
                for parameter in name_chld.parameters():
                    parameter.requires_grad = False  # Freeze the layer
            else:
                for parameter in name_chld.parameters():
                    parameter.requires_grad = True  # Make the layer trainable
            count += 1
    return pretrained_model

# Example usage
freeze_percent = 0.25  # Freeze 25% of layers
freez_all_except_last_layer = 'No'
model_name = 'resnet'
model = pretrain_model(model_name,freeze_percent,freez_all_except_last_layer)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:

data_dir = '/kaggle/input/nature-12k-dataset/inaturalist_12K/train'

def data_load(data_dir,data_augumentation):
    # Define data transformations
    if data_augumentation == 'Yes':
        transform = transforms.Compose([
            transforms.RandomResizedCrop(224),  # Randomly crop and resize to 224x224
            transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
            transforms.RandomRotation(20),  # Randomly rotate the image by up to 20 degrees
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image tensors
        ])

    else:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize images to 224x224 (compatible with the CNN input size)
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image tensors
        ])
        
    # Load the dataset using ImageFolder and apply transformations
    training_data = ImageFolder(root=data_dir, transform=transform)
    # Splitting train dataset into training and validation indices
    train_index, val_index = train_test_split(list(range(len(training_data))), test_size=0.2, random_state=42)
    # Create DataLoader instances for training and validation sets
    random_train_sample = SubsetRandomSampler(train_index)
    train_data = DataLoader(training_data, batch_size=32, sampler=random_train_sample)
    random_val_sample = SubsetRandomSampler(val_index)
    validation_data = DataLoader(training_data, batch_size=32, sampler=random_val_sample)
    return train_data, validation_data

In [16]:
def train_on_train_data(model, train_data):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.train()  # Set the model to training mode
    training_loss = 0.0
    correct_train_label = 0
    total_train = 0
    # Training loop
    for inputs, labels in train_data:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()

        # Calculate training accuracy
        _, pred = torch.max(outputs, 1)
        correct_train_label += (pred == labels).sum().item()
        total_train += labels.size(0)

    # Calculate training loss and accuracy
    avg_loss = training_loss / len(train_data)
    train_accuracy = 100 * correct_train_label / total_train
    return avg_loss, train_accuracy

    

In [12]:
def test_on_valid_data(model, test_data):
    model.eval()  # Set the model to evaluation mode
    correct_val = 0
    total_val = 0
    with torch.no_grad():  # Turn off gradient calculation for validation
        for inputs_val, labels_val in test_data:
            inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)
            outputs_val = model(inputs_val)
            _, predicted_val = torch.max(outputs_val, 1)
            correct_val += (predicted_val == labels_val).sum().item()
            total_val += labels_val.size(0)

    # Calculate validation accuracy
    valid_accuracy = 100 * correct_val / total_val
    return valid_accuracy

In [13]:
# Function to training model
def model_train(model, train_data, val_data,epochs):

    criterion = nn.CrossEntropyLoss()
    # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        avg_loss, train_accuracy = train_on_train_data(model, train_data)
        # Print training loss and accuracy
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        wandb.log({'Train loss': avg_loss})
        wandb.log({'Train accuracy': train_accuracy})


        # Validation loop
        val_accuracy = test_on_valid_data(model, val_data)
        # Print validation accuracy
        print(f'Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%')
        wandb.log({'val_accuracy': val_accuracy})
        wandb.log({'epoch': epoch})

    print('Training complete!')

# Example usage
# model_train(model, train_loader, val_loader)


In [14]:
!pip install wandb

In [ ]:


import wandb
import numpy as np
from types import SimpleNamespace
import random

wandb.login(key='bb3c7761be2856a8335d16d1483149380482ae9e')#bb3c7761be2856a8335d16d1483149380482ae9e

sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epoch': {
            'values': [6]
        },
        'freezing_percentage': {
            'values': [0.8,0.9]
        },
        'model': {
            'values': [ 'resnet']#,'inception']
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='DL_assignment_2_PartB')


def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''

    with wandb.init() as run:
        run_name="ep"+str(wandb.config.epoch)+"_fp-"+str(wandb.config.freezing_percentage)+"_model-"+(wandb.config.model)
        wandb.run.name=run_name
# wandb.config.freezing_percentage   "_fp-"+str(wandb.config.freezing_percentage)+

        model = pretrain_model(model_name = wandb.config.model,freeze_percent=wandb.config.freezing_percentage ,freez_all_except_last_layer='No')
        model = model.to(device)

        data_dir = '/kaggle/input/nature-12k-dataset/inaturalist_12K/train'
        train, validation = data_load(data_dir,data_augumentation= 'No')

        model_train(model,train,validation,epochs=wandb.config.epoch)

wandb.agent(sweep_id, function= main,count= 1) # calls main function for count number of times.
wandb.finish()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: zffq76ui
Sweep URL: https://wandb.ai/abanisingha1997/DL_assignment_2_PartB/sweeps/zffq76ui


wandb: Agent Starting Run: 8bkdx1xy with config:
wandb: 	epoch: 6
wandb: 	freezing_percentage: 0.9
wandb: 	model: resnet


Epoch 1/6, Train Loss: 1.1011, Train Accuracy: 65.27%
Epoch 1/6, Validation Accuracy: 72.75%
